In [1]:
using LinearAlgebra


In [ ]:
# Fonction pour résoudre la relaxation LP avec Simplex (ou autre méthode LP)
function solve_relaxed_lp_bis(A, b, c)
    # Nombre de contraintes (m) et de variables (n)
    m, n = size(A)

    # Création du tableau Simplex
    # On ajoute les variables d'écart (identité de taille m) et la colonne des b
    tableau = hcat(A, I(m), b)  # Ajout des variables d'écart et des termes constants
    tableau = vcat(tableau, hcat(c', zeros(1, m + 1)))  # Ajout de la ligne de la fonction objectif

     # Déclaration du vecteur `base`, qui stocke les indices de base
    base = Vector{Int}(undef, m)  # Crée un vecteur non initialisé de taille m
    
    # Initialisation de base avec les valeurs des variables d'écart
    for i in 1:m
        base[i] = n + i  # Assignation des indices des variables d'écart
    end

    # Boucle principale du Simplex
    while true
        # Vérifier les coûts réduits (ligne de la fonction objectif)
        costs = tableau[end, 1:n+m]
        
        # Si tous les coûts réduits sont <= 0, on a une solution optimale
        if all(costs .<= 0)
            break
        end

        # Trouver la variable entrante (la colonne avec le coût réduit le plus élevé)
        entering_var = argmax(costs)

        # Calcul des rapports pour déterminer la variable sortante
        ratios = []
        for i in 1:m
            if tableau[i, entering_var] > 0
                push!(ratios, tableau[i, end] / tableau[i, entering_var])
            else
                push!(ratios, Inf)  # Si le pivot est <= 0, ignorer cette ligne
            end
        end

        # Trouver la variable sortante
        leaving_var = argmin(ratios)

        # Mettre à jour les indices de base
        base[leaving_var] = entering_var

        # Mise à jour du tableau Simplex (pivotage)
        pivot = tableau[leaving_var, entering_var]
        tableau[leaving_var, :] ./= pivot  # Normaliser la ligne pivot

        for i in 1:m+1
            if i != leaving_var
                tableau[i, :] .-= tableau[i, entering_var] .* tableau[leaving_var, :]
            end
        end
    end

    # Extraire la solution optimale
    solution = zeros(n)
    for i in 1:m
        if base[i] <= n  # Si la variable de base est une variable originale
            solution[base[i]] = tableau[i, end]
        end
    end

    return solution
end


In [6]:
function solve_relaxed_lp(A, b, c)
    # Nombre de contraintes (m) et de variables (n)
    m, n = size(A)

    # Création du tableau Simplex
    tableau = hcat(A, I(m), b)  # Ajout des variables d'écart et des termes constants
    tableau = vcat(tableau, hcat(c', zeros(1, m + 1)))  # Ajout de la ligne de la fonction objectif

    # Déclaration du vecteur `base`
    base = Vector{Int}(undef, m)
    for i in 1:m
        base[i] = n + i  # Indices des variables d'écart
    end

    max_iterations = 1000  # Limite d'itérations pour éviter la boucle infinie
    iteration = 0

    # Boucle principale du Simplex
    while true
        costs = tableau[end, 1:n+m]
        
        # Condition de sortie si tous les coûts réduits sont <= 0
        if all(costs .<= 0)
            break
        end

        # Trouver la variable entrante
        entering_var = argmax(costs)

        # Calcul des rapports pour déterminer la variable sortante
        ratios = [tableau[i, end] / tableau[i, entering_var] for i in 1:m if tableau[i, entering_var] > 0]

        # Si tous les ratios sont infinis, il n'y a pas de variable sortante
        if isempty(ratios)
            println("Pas de variable sortante trouvée, solution non bornée.")
            break
        end

        # Trouver la variable sortante
        leaving_var = argmin(ratios)

        # Mettre à jour les indices de base
        base[leaving_var] = entering_var

        # Mise à jour du tableau
        pivot = tableau[leaving_var, entering_var]
        tableau[leaving_var, :] ./= pivot

        for i in 1:m+1
            if i != leaving_var
                tableau[i, :] .-= tableau[i, entering_var] .* tableau[leaving_var, :]
            end
        end

        # Vérifier si nous avons atteint le nombre maximum d'itérations
        iteration += 1
        if iteration > max_iterations
            println("Atteint la limite d'itérations sans trouver de solution.")
            break
        end
    end

    # Extraire la solution optimale
    solution = zeros(n)
    for i in 1:m
        if base[i] <= n
            solution[base[i]] = tableau[i, end]
        end
    end

    return solution
end


solve_relaxed_lp (generic function with 1 method)

In [7]:
# Fonction de branchement et de coupe (branch and cut)
function branch_and_cut(A, b, c, variables, depth = 0)
    # Résoudre le problème relaxé
    x_relaxed = solve_relaxed_lp(A, b, c)

    # Vérifier si la solution est entière
    if all(x -> isinteger(x), x_relaxed)
        println("Solution entière trouvée : ", x_relaxed)
        return x_relaxed
    end

    # Si la solution n'est pas entière, on branche sur une variable fractionnaire
    for i in 1:length(x_relaxed)
        if x_relaxed[i] ≠ 0 && x_relaxed[i] ≠ 1
            #println("Branching on variable $i at depth $depth")

            # Créer deux sous-problèmes : un avec x[i] = 0, l'autre avec x[i] = 1
            # Branch 1: Ajouter la contrainte x[i] = 0
            new_variables_zero = copy(variables)
            new_variables_zero[i] = (0, 0)
            solution_zero = branch_and_cut(A, b, c, new_variables_zero, depth + 1)

            # Branch 2: Ajouter la contrainte x[i] = 1
            new_variables_one = copy(variables)
            new_variables_one[i] = (1, 1)
            solution_one = branch_and_cut(A, b, c, new_variables_one, depth + 1)

            # Comparer les solutions trouvées dans les deux branches
            if solution_zero !== nothing && solution_one !== nothing
                return norm(c * solution_zero) < norm(c * solution_one) ? solution_zero : solution_one
            elseif solution_zero !== nothing
                return solution_zero
            elseif solution_one !== nothing
                return solution_one
            else
                return nothing  # Aucun résultat valide trouvé
            end
        end
    end
end




branch_and_cut (generic function with 2 methods)

In [8]:
function setup_constraints(num_tasks, num_machines, resources)
    A = zeros(num_machines, num_tasks*num_machines)  # Matrice de contraintes
    b = zeros(num_tasks*num_machines)  # Vecteur des bornes

    # Contraintes CPU
    for j in 1:num_machines
        for i in 1:num_tasks
            A[j, (j-1)*num_tasks + i] = resources[:cpu_req][i]
        end
        b[j] = resources[:cpu_dispo][j]
    end

    # Contraintes RAM
    for j in 1:num_machines
        for i in 1:num_tasks
            A[j+num_machines, (j-1)*num_tasks + i] = resources[:ram_req][i]
        end
        b[j+num_machines] = resources[:ram_dispo][j]
    end

    # Contraintes Threads, Bandwidth, IO, Température - Idem

    return A,b 
end

setup_constraints (generic function with 1 method)

In [9]:
function build_lp_matrices(num_tasks, num_machines, resources)
    num_variables = num_tasks * num_machines  # Nombre total de variables de décision
    num_constraints = 6 * num_machines  # 6 contraintes (CPU, RAM, Threads, BW, IO, Température) par machine

    # Initialisation de A (num_constraints x num_variables) et b (num_constraints)
    A = zeros(num_constraints, num_variables)
    b = zeros(num_constraints)

    # Ajout des contraintes pour chaque machine
    for j in 1:num_machines
        # Indices des lignes correspondant à cette machine
        row_base = (j - 1) * 6  # Chaque machine a 6 contraintes, donc on se déplace de 6 lignes à chaque fois
        
        # CPU constraint pour la machine j
        for i in 1:num_tasks
            A[row_base + 1, (j - 1) * num_tasks + i] = resources[:cpu_req][i]
        end
        b[row_base + 1] = resources[:cpu_dispo][j]

        # RAM constraint pour la machine j
        for i in 1:num_tasks
            A[row_base + 2, (j - 1) * num_tasks + i] = resources[:ram_req][i]
        end
        b[row_base + 2] = resources[:ram_dispo][j]

        # Threads constraint pour la machine j
        for i in 1:num_tasks
            A[row_base + 3, (j - 1) * num_tasks + i] = resources[:threads_req][i]
        end
        b[row_base + 3] = resources[:threads_dispo][j]

        # Bande passante (BW) constraint pour la machine j
        for i in 1:num_tasks
            A[row_base + 4, (j - 1) * num_tasks + i] = resources[:bw_req][i]
        end
        b[row_base + 4] = resources[:bw_dispo][j]

        # IO constraint pour la machine j
        for i in 1:num_tasks
            A[row_base + 5, (j - 1) * num_tasks + i] = resources[:io_req][i]
        end
        b[row_base + 5] = resources[:io_dispo][j]

        # Température constraint pour la machine j
        for i in 1:num_tasks
            A[row_base + 6, (j - 1) * num_tasks + i] = resources[:temp_req][i]
        end
        b[row_base + 6] = resources[:temp_max][j] - resources[:temp_current][j]
    end

    # Vecteur objectif c pour maximiser l'utilisation des ressources
    c = zeros(num_variables)
    for j in 1:num_machines
        for i in 1:num_tasks
            c[(j - 1) * num_tasks + i] = resources[:cpu_req][i] + resources[:ram_req][i] + resources[:threads_req][i] + resources[:bw_req][i] + resources[:io_req][i]
        end
    end

    return A, b, c
end

build_lp_matrices (generic function with 1 method)

In [10]:
# Fonction principale pour résoudre le problème
function solve(num_tasks, num_machines, resources, exec_time)
    # A, b = setup_constraints(num_tasks, num_machines, resources)
    # println(size(hcat(A)))
    # c = vec(exec_time)  # Vectorisation des temps d'exécution
    A, b, c = build_lp_matrices(num_tasks, num_machines, resources)
    println(size(b))
    # Variables initiales avec des bornes (0, 1) pour chaque tâche sur chaque machine
    variables = [(0, 1) for _ in 1:num_tasks * num_machines]

    # Résoudre avec Branch and Cut
    solution = branch_and_cut(A, b, c, variables)

    return solution
end

solve (generic function with 1 method)

In [11]:

# Exemple d'utilisation
num_tasks = 3
num_machines = 2
resources = Dict(
    :cpu_req => [2, 3, 1],
    :ram_req => [1, 2, 1],
    :threads_req => [2, 3, 2],
    :bw_req => [10, 15, 5],
    :io_req => [100, 200, 50],
    :temp_req => [10, 15, 5],
    :cpu_dispo => [6, 8],
    :ram_dispo => [8, 10],
    :threads_dispo => [5, 6],
    :bw_dispo => [50, 70],
    :io_dispo => [400, 500],
    :temp_max => [90, 100],
    :temp_current => [30, 40]
)
exec_time = [5 3; 2 6; 4 2]

# Résoudre
solution = solve(num_tasks, num_machines, resources, exec_time)
if solution !== nothing
    println("Meilleure solution trouvée: ", solution)
else
    println("Aucune solution trouvée.")
end

(12,)
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas 

Excessive output truncated after 524291 bytes.


Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de variable sortante trouvée, solution non bornée.
Pas de va

LoadError: StackOverflowError:

In [79]:
m,n = size(A)
base = n + 1:n + m

7:18

6